In [17]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_core.tools import Tool
from IPython.display import display, Markdown

from dotenv import load_dotenv

load_dotenv(override=True)

True

In [18]:
ALL_IN_ONE_WORKER = True

In [19]:
@dataclass
class Message:
    content: str

To import GrpcWorkerAgentRuntimeHost, we need to install the following
```bash
uv add autogen-ext[grpc]
```

In [20]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start() 

In [21]:
serper = GoogleSerperAPIWrapper()
langchain_serper =Tool(name="internet_search", func=serper.run, description="Useful for when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)

In [22]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

In [23]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client)
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)

In [24]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

if ALL_IN_ONE_WORKER:

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId("judge", "default")

else:

    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")


In [25]:
response = await worker.send_message(Message(content="Go!"), agent_id)

In [26]:
display(Markdown(response.content))

## Pros of AutoGen:
Here are some pros of using AutoGen for your AI Agent project:

1. **Multi-agent Capability**: AutoGen supports the creation of multi-agent systems, which can outperform single-agent configurations in various tasks, enhancing performance in applications like math problem-solving and question answering.

2. **Simplified Orchestration**: The framework abstracts away the complexity of managing agent interactions, making it easier for developers to build and deploy complex workflows with multiple agents.

3. **Advanced AI Capabilities**: AutoGen is designed for complex automation and dynamic environments, making it suitable for tasks that require adaptable AI responses.

4. **High Customization**: It allows compatibility with various large language models (LLMs) and tooling, enabling the creation of tailored solutions for different tasks.

5. **Modular Architecture**: The separation of perception, reasoning, and action modules provides flexibility and a clearer development process for creating sophisticated AI applications.

6. **Event-driven Design**: This feature enhances responsiveness and scalability, enabling agents to quickly react to changes in their environments.

Overall, AutoGen presents a robust framework that can facilitate the development of sophisticated and effective AI solutions. 

TERMINATE

## Cons of AutoGen:
Here are some cons against choosing AutoGen for an AI Agent project:

1. **Limited AI Capabilities**: AutoGen may not perform well with complex AI-driven automation tasks, potentially limiting its applicability in advanced scenarios.

2. **Less Customizable**: The framework offers limited flexibility for custom integrations, which can hinder the ability to tailor solutions to specific project needs.

3. **Less Structured**: Compared to other frameworks, AutoGen may be perceived as less structured, which can complicate implementation efforts and lead to potential confusion during development.

4. **Complexity of Prompts**: It requires thorough and sophisticated algorithmic prompts, which can be time-consuming and costly to develop.

5. **High Maintenance Requirements**: Projects utilizing AutoGen may face higher maintenance needs, demanding significant human oversight and resources.

6. **Unpredictability**: Agentic AI frameworks, like AutoGen, can exhibit unpredictable behaviors, which might compromise reliability and safety in critical applications.

These factors should be carefully considered when deciding whether to use AutoGen for your AI Agent project. 

TERMINATE



## Decision:

Based on the research provided by the team, I recommend using AutoGen for the project. 

The robust advantages of AutoGen, including its multi-agent capability, simplified orchestration, advanced AI functionalities, high customization, and modular architecture, can significantly enhance the performance and effectiveness of the AI solutions we aim to develop. Although there are valid concerns regarding its limitations in certain complex tasks, structuredness, and maintenance requirements, the pros present a strong case for its potential to facilitate innovative and adaptable AI applications. The framework's ability to handle complex workflows and event-driven designs aligns with our project's needs for flexibility and responsiveness.

TERMINATE

In [27]:
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

In [28]:
await host.stop()